# Emotion Detection Model generation

Our Futures project's goal is to create an empathetic, predicitve system for people with dementia. Hence, we want to use emotional data to help support our predicitve system which is why prototyping an emotion recognition algorithm was essential. For this project a Convolutional Neural Network is built from scratch by using Keras. To understand this thouroughly you should have some basic knowledge of python, Convolutional Neural Network(CNN) and the different layers which are used in CNN. The notebook is divided into tasks to make it easier to understand.

## Task 1:

Import the required modules that are needed in this project.

In [18]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
import os

Now let's define some variables that will save the time of writing the values manually again and again.

- num_classes:  defines the number of outputs (emotions) that the algorithm can give
- img_rows,img_cols: defines the size of the array fed to the NN (image size)
- batch_size: defines the batch size which is the number of samples processed before the model is updated. The number of epochs is the number of complete passes through the training dataset. The size of a batch must be more than or equal to one and less than or equal to the number of samples in the training dataset.

In [1]:
num_classes=5 
img_rows,img_cols=48,48 
batch_size=32 

## Task 2:

Now it's time to load our dataset (**fer2013**) dataset which is  open source and hosted on [kaggle](https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data). The data is segragated as training and validation set.

The lines below import the validation and training data from local directory.

In [2]:
train_data_dir='fer2013/train'
validation_data_dir='fer2013/validation'

## Task 3:

Image Augmentation techniques are used on the dataset. This is a technique that can be used for artificial expansion of the size of a training dataset by creating modified versions of images. Keras provides the capability to fit models using image data augmentation via the ImageDataGenerator class.

The train_datagen variable expands the dataset using the following:

- rotation_range: degree range for random rotations (30$^\circ$)
- shear_range: shear intensity (counter-clockwise direction)
- zoom_range: range for random zoom (0.3)
- width_shift_range: shifts the images by a value across its width
- height_shift_range : shifts the images by a value across its height
- horizontal_flip: flips the images horizontally
- fill_mode: used to fill in the pixels after making changes to the orientation of the images by the above used methods. 'nearest' is used as the fill mode instructing it to fill the missing pixels in the image with the nearby pixels.

In [21]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    shear_range=0.3,
    zoom_range=0.3,
    width_shift_range=0.4,
    height_shift_range=0.4,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

The **flow_from_directory()** method is used to load the dataset from the directory which is augmented and stored in the train_generator and validation_generator variables.**flow_from_directory()**  takes the path to a directory & generates batches of augmented data. 

The options given are:

- directory:  directory of the dataset.
- color_mode:  converting the images to grayscale as I am not interested in the color of the images but only the expressions.
- target_size: converting the images to a uniform size.
- batch_size:  make baches of data to train.
- class_mode: 'categorical' is used as the class mode because the images are assigned to 5 classes.
- shuffle: shuffles the dataset for better training.

In [22]:
train_generator = train_datagen.flow_from_directory(
                        train_data_dir,
                        color_mode='grayscale',
                        target_size=(img_rows,img_cols),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
                                validation_data_dir,
                                color_mode='grayscale',
                                target_size=(img_rows,img_cols),
                                batch_size=batch_size,
                                class_mode='categorical',
                                shuffle=True)

Found 24256 images belonging to 5 classes.
Found 3006 images belonging to 5 classes.


## Task 4:

Now that the dataset pre-processing is complete, we can construct the CNN to create this DL model. A **Sequential** model is used as it  defines that all the layers in the network will be one after the other sequentially.

In [23]:
model = Sequential()

The network consists of 7 blocks which will be explained below.  

Here i have used the layers of 7 types which are present in **keras.layers**.

The layers are:

- Conv2D(
    filters, kernel_size, strides=(1, 1), padding='valid', data_format=None,
    dilation_rate=(1, 1), activation=None, use_bias=True,
    kernel_initializer='glorot_uniform', bias_initializer='zeros',
    kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None,
    kernel_constraint=None, bias_constraint=None, **kwargs
)
- Activation(activation_type)

- BatchNormalization()

- MaxPooling2D(pool_size, strides, padding, data_format, **kwargs)

- Dropout(dropout_value) 

- Flatten()

- Dense(
    units,
    activation=None,
    use_bias=True,
    kernel_initializer="glorot_uniform",
    bias_initializer="zeros",
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
    **kwargs)

#### Block-1 layers in the order of occurrence are as follows :

- Conv2D layer-  creates a convolutional layer for the network. The layer is created with 32 filters and a filter size of (3,3) with padding='same' to pad the image and using the kernel initializer he_normal. I have added 2 convolutional layers each followed by an activation and batch normalization layers.
- Activation layer - elu activation is being used
- BatchNormalization - normalize the activations of the previous layer at each batch (applies a transformation that maintains the mean activation close to 0 and the activation standard deviation close to 1)
- MaxPooling2D layer - downsamples the input representation by taking the maximum value over the window defined by pool_size for each dimension along the features axis (pool_size of (2,2))
- Dropout: technique where randomly selected neurons are ignored during training. A dropout of 0.5 is used which means that it will ignore half of the neurons.

#### Block-2 layers in the order of occurrence are as follows :

- Same layers as block-1 but the convolutional layers have 64 filters.

#### Block-3 layers in the order of occurrence are as follows :

- Same layers as block-1 but the convolutional layers have 128 filters.

#### Block-4 layers in the order of occurrence are as follows :

- Same layers as block-1 but the convolutional layers have 256 filters.

#### Block-5 layers in the order of occurrence are as follows :

- Flatten layer - flattens the output of the previous layers in a fallout layer or in other words in the form of a vector.
- Dense layer - densely connected layer where each neuron is connected to every other neuron (64 neurons with a kernal initializer - he_normal)
- These layers are followed by activation layer with elu activation , batch normalization and finally a dropout with 50% dropout.

#### Block-6 layers in the order of occurrence are as follows :

- Same layers as block 5 but without flatten layer as the input for this block is already flattened.

#### Block-7 layers in the order of occurrence are as follows :

- Dense layer - Finally in the final block of the network, num_classes is used to create a dense layer having the same number of units as there are classes with a he_normal initializer.

- Activation layer - using a softmax layer which is used for multi-class classifications.

- ### Block-1

In [24]:
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

- ### Block-2

In [25]:
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

- ### Block-3

In [26]:
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

- ### Block-4

In [27]:
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

- ### Block-5

In [28]:
model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

- ### Block-6

In [29]:
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

- ### Block-7

In [30]:
model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

Now to check the overall structure of the model:

In [31]:
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 48, 48, 32)        320       
_________________________________________________________________
activation_12 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_11 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_13 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 24, 24, 32)       

This is a big network which consits of 1,328,037 parameters.

## Task 5:

Now the only thing left is to compile and train the model. But first let's import some more things.

In [32]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

Now let's do the magic.

Before compiling 3 callbacks are created to modify the code while it is compiling if needed. This is done using **keras.callbacks** class:

#### Checkpoint( Function - ModelCheckpoint() )

It will monitor the validation loss and will try to minimise the loss using the mode='min' property. When the checkpoint is reached it will save the best trained weights. Verbose=1 is just for visulaisation when the code created checkpoint. The following parametrs are being used:

- filepath: path to save the model file (model file with the name EmotionDetectionModel.h5)
- monitor: quantity to monitor (validation loss)
- mode: one of {auto, min, max}. If save_best_only=True, the decision to overwrite the current save file is made based on either the maximization or the minimization of the monitored quantity.
- save_best_only: if save_best_only=True, the latest best model according to the quantity monitored will not be overwritten.
- verbose: int. 0: quiet, 1: update messages.

#### Early Stopping ( Function - EarlyStopping() )

This will stop the execution early by checking the following properties.

- monitor:  quantity to monitor (validation loss).
- min_delta: Minimum change in the monitored quantity to qualify as an improvement, absolute change of less than min_delta, will count as no improvement (assigned a value of 0)
- patience: number of epochs with no improvement after which training will be stopped (value of 3)
- restore_best_weights: Whether to restore model weights from the epoch with the best value of the monitored quantity. If False, the model weights obtained at the last step of training are used (assigned True)
- verbose: int. 0: quiet, 1: update messages.

#### Reduce Learning Rate ( Function - ReduceLROnPlateau() )

Models often benefit from reducing the learning rate by a factor of 2-10 once learning stagnates. This callback monitors a quantity and if no improvement is seen for a 'patience' number of epochs, the learning rate is reduced. The following properties are used:

- monitor: to monitor a particular loss (validation loss here)
- factor: factor by which the learning rate will be reduced. **new_lr = lr * factor**. (using 0.2 as factor).
- patience: Number of epochs with no improvement after which learning rate will be reduced (initial value of 3)
- min_delta: threshold for measuring the new optimum, to only focus on significant changes.
- verbose: int. 0: quiet, 1: update messages.

In [33]:

checkpoint = ModelCheckpoint('EmotionDetectionModel.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop,checkpoint,reduce_lr]

Now it's time to finally compile the model using **model.compile()** and fit or train the model on the dataset using **model.fit_generator()**

#### model.compile()

It has the following arguments:

- loss: will determine the type of loss function to use in your code. We have categorical data in 5 categories or classes so 'categorical_crossentropy' loss is used.
- optimizer: will determine the type of optimizer function to use in your code. Adam optimizer with learning rate 0.001 is used as it is the best optimizer for categorical data.
- metrics: list of metrics to be evaluated by the model during training and testing. Here, accuracy is used as metric which will compile mu model according to the accuracy.

#### model.fit_generator()

Fits the model on data yielded batch-by-batch by a Python generator.

It has the following arguments:

- generator: The train_generator object that we created earlier.
- steps_per_epochs: The steps to take on the training data in one epoch.
- epochs: The total number of epochs (pass though the whole dataset once).
- callbacks: The list containing all the callbacks that we created earlier.
- validation_data: The validation_generator object that we created earlier.
- validation_steps: The steps to take on the validation data in one epoch.

In [34]:
model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])

nb_train_samples = 24176
nb_validation_samples = 3006
epochs=25

history=model.fit_generator(
                train_generator,
                steps_per_epoch=nb_train_samples//batch_size,
                epochs=epochs,
                callbacks=callbacks,
                validation_data=validation_generator,
                validation_steps=nb_validation_samples//batch_size)

Epoch 1/25
755/755 [==============================] - 266s 353ms/step - loss: 1.8671 - accuracy: 0.2326 - val_loss: 1.6492 - val_accuracy: 0.2991

Epoch 00001: val_loss improved from inf to 1.64916, saving model to EmotionDetectionModel.h5
Epoch 2/25
755/755 [==============================] - 275s 364ms/step - loss: 1.5839 - accuracy: 0.2759 - val_loss: 1.5460 - val_accuracy: 0.2946

Epoch 00002: val_loss improved from 1.64916 to 1.54597, saving model to EmotionDetectionModel.h5
Epoch 3/25
755/755 [==============================] - 537s 711ms/step - loss: 1.5494 - accuracy: 0.3025 - val_loss: 1.5584 - val_accuracy: 0.3067

Epoch 00003: val_loss did not improve from 1.54597
Epoch 4/25
755/755 [==============================] - 377s 499ms/step - loss: 1.5322 - accuracy: 0.3136 - val_loss: 1.4969 - val_accuracy: 0.3564

Epoch 00004: val_loss improved from 1.54597 to 1.49691, saving model to EmotionDetectionModel.h5
Epoch 5/25
755/755 [==============================] - 214s 284ms/step - lo